In [11]:
from google.colab import files
uploaded = files.upload()


Saving X.npy to X (1).npy


In [12]:
from google.colab import files
uploaded = files.upload()

Saving Y.npy to Y (1).npy


In [21]:
# Breast Histology Images
# Classify IDC vs non IDC images
#
# https://www.kaggle.com/simjeg/lymphoma-subtype-classification-fl-vs-cll
#
# This dataset consists of 5547 breast histology images of size 50 x 50 x 3,
# The goal is to classify cancerous images (IDC : invasive ductal carcinoma) vs non-IDC images.
#
# Perceptron
#


import tensorflow as tf
import numpy as np
import time
import os
from sklearn.model_selection import train_test_split


def print_time(message, start_time):
    print("*****   " + message + ": {}  *****".format(time.time() - start_time))


def get_batch(all_data, all_labels, batch_size=16):
    # // means divide and result is integer, / returns float
    # rows_data = len(all_data) // batch_size
    # rows_labels = len(all_labels) // batch_size

    rtn_data = all_data.reshape(-1, batch_size, int(50*50*3))
    rnt_labels = all_labels.reshape(-1, batch_size, 2)

    return (rtn_data, rnt_labels)


def multiperceptron(x):
    # tf.nn.sigmoid() 1/(1+e^-x)
    # x[16, 3072], l1[16,256], l2[16, 256], out[16, 10]
    l1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['h1']), biases['b1']))
    l2 = tf.nn.sigmoid(tf.add(tf.matmul(l1, weights['h2']), biases['b2']))
    l3 = tf.nn.sigmoid(tf.add(tf.matmul(l2, weights['h3']), biases['b3']))
    outl = tf.nn.sigmoid(tf.add(tf.matmul(l3, weights['out']), biases['out']))
    return outl

#####################################################################################################
#
#   Main
#
#
######################################################################################################
START_TIME = time.time()
path = r'C:\Users\gtune\OneDrive\document_usk\UCSC\02_SecondQuater\Deep_Learning_and_Artificial_Intelligence_with_TensorFlow\Homework\Final_Project\data'

X_data = np.load("X.npy")
Y_truth = np.load("Y.npy")

# print(X_data[0:10])
# print(X_data.shape)
# print(Y_truth[0:100])

# print(np.array(np.where(Y_truth == 1)))

train_data, test_data, train_Y, test_Y = \
    train_test_split(X_data, Y_truth, test_size=0.25, random_state=3)

print(len(np.array(np.where(train_Y == 1)).ravel()))
print(len(np.array(np.where(train_Y == 0)).ravel()))

print(train_data.shape)
print(train_Y.shape)
#print(train_Y[0:50])

# architecture hyper-parameter
num_datapoints = len(train_data)
learning_rate = 0.0001   # 0.001
n_epoch = 1000
batch_size = 52  # divisor of 4160 : 1, 2, 4, 5, 8, 10, 13, 16, 20, 26, 32, 40, 52, 64, 65, 80, 104, 130, 160, 208, 260, 320, 416, 520, 832, 1040, 2080, 4160

n_input = 50*50*3
n_classes = 2   # IDC and non-IDC
dropout = 0.75

# Normalization
max_value = np.max(train_data)
print('max_value -> {}'.format(max_value))

train_data = np.array(train_data/max_value, dtype=np.float32)

# data and Y(Label) reshaped
onehot_train_Y = np.full((len(train_Y), n_classes), 0)
onehot_train_Y[np.arange(0, len(train_Y)), train_Y] = 1
train_data, onehot_train_Y = get_batch(train_data, onehot_train_Y, batch_size)

test_data = test_data.reshape(-1, n_input)
onehot_test_Y = np.full((len(test_Y), n_classes), 0)
onehot_test_Y[np.arange(0, len(test_Y)), test_Y] = 1


num_batches = num_datapoints // batch_size  # 10  # num_datapoints // batch_size

print_time("num_batches", START_TIME)

# data_x, labels_y = get_batch(X_train, onehot_labels, batch_size)

print("NO OF BATCHES:", num_batches)
print_time("NO OF BATCHES", START_TIME)


# tensorflow placeholder
X = tf.placeholder(tf.float32, [None, n_input])
Y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

n_hidden1 = 1024
n_hidden2 = 512
n_hidden3 = 256

weights = \
{
        'h1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'h2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'h3': tf.Variable(tf.random_normal([n_hidden2, n_hidden3])),
        'out': tf.Variable(tf.random_normal([n_hidden3, n_classes]))
}

# b1=[512], b2=[256], b3=[128], out=[10]
biases = \
{
    'b1': tf.Variable(tf.random_normal([n_hidden1])),
    'b2': tf.Variable(tf.random_normal([n_hidden2])),
    'b3': tf.Variable(tf.random_normal([n_hidden3])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}
# Create the Model
model = multiperceptron(X)

# Define loss and optimizer
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
train_min = tf.train.AdamOptimizer(learning_rate).minimize(loss)

# Evaluate the model
correct_model = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_model, tf.float32))

# Initialing the variables
init = tf.global_variables_initializer()

# Tensorflow Session

with tf.Session() as sess:

    sess.run(init)

    for epoch in range(n_epoch):
        for i in range(num_batches):
            # for _ in range(1):

            batch_x = train_data[i]
            batch_y = onehot_train_Y[i]

            # Use training data for optimization
            sess.run(train_min, feed_dict={X: batch_x, Y: batch_y, keep_prob: dropout})

        # varidate after every epoch
        batch_x = train_data[0]
        batch_y = onehot_train_Y[0]

        losscalc, accuracycalc = sess.run([loss, accuracy], feed_dict={X:batch_x, Y:batch_y, keep_prob:1.0})

        test_accuracycalc = sess.run(accuracy, feed_dict={X: test_data, Y: onehot_test_Y, keep_prob: 1.0})

        print("Epoch: %d, Loss: %0.4f, Train_Acc: %0.4f, TEST_Acc: %0.4f, Time: %0.4f" % (epoch, losscalc, accuracycalc, test_accuracycalc, time.time() - START_TIME))

        # when train accuracy is over 95%, program end
        if accuracycalc >= 0.95:
            break

    # display the accuracy of using testing data
    accuracycalc = sess.run(accuracy, feed_dict={X: test_data, Y: onehot_test_Y, keep_prob: 1.0})

    print("                  Testing accuracy: %0.4f, Time: %0.4f" % (accuracycalc, time.time() - START_TIME))

print_time("END", START_TIME)



2078
2082
(4160, 50, 50, 3)
(4160,)
max_value -> 255
*****   num_batches: 0.22154831886291504  *****
NO OF BATCHES: 80
*****   NO OF BATCHES: 0.22242164611816406  *****
Epoch: 0, Loss: 0.7046, Train_Acc: 0.5962, TEST_Acc: 0.5804, Time: 4.8368
Epoch: 1, Loss: 0.6549, Train_Acc: 0.5577, TEST_Acc: 0.6006, Time: 5.4109
Epoch: 2, Loss: 0.6544, Train_Acc: 0.5962, TEST_Acc: 0.6352, Time: 5.9690
Epoch: 3, Loss: 0.6718, Train_Acc: 0.6154, TEST_Acc: 0.6547, Time: 6.5241
Epoch: 4, Loss: 0.6429, Train_Acc: 0.5577, TEST_Acc: 0.6619, Time: 7.0823
Epoch: 5, Loss: 0.6143, Train_Acc: 0.5577, TEST_Acc: 0.6633, Time: 7.6344
Epoch: 6, Loss: 0.6257, Train_Acc: 0.6731, TEST_Acc: 0.6655, Time: 8.1951
Epoch: 7, Loss: 0.6289, Train_Acc: 0.5962, TEST_Acc: 0.6698, Time: 8.7494
Epoch: 8, Loss: 0.5971, Train_Acc: 0.6923, TEST_Acc: 0.6633, Time: 9.3068
Epoch: 9, Loss: 0.6343, Train_Acc: 0.6538, TEST_Acc: 0.6878, Time: 9.8610
Epoch: 10, Loss: 0.6161, Train_Acc: 0.6346, TEST_Acc: 0.6972, Time: 10.4359
Epoch: 11, Loss